


### Semantic Segmentation Of Images to calculate the cloud cover percentage, sky and Sun positions.




In [ ]:
# Function to prevent idleness of the Collab Notebook
"""
function ConnectButton(){
  console.log("Connect pushed");
  document.querySelector("#top-toolbar > colab-connectbutton").shadowRoot.querySelector("#connect").click()
}
setInterval(ConnectButton,60000);
"""

In [ ]:
#importing necessary libraries

import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
from datetime import datetime
import threading

In [ ]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Finding Sun in the picture

In [ ]:

def find_sun(image_path):
    image = cv2.imread(image_path)

    # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # # plt.title('Processed Image with Sun Detection')
    # plt.axis('off')
    # plt.show()

    if image is None:
        print("Error: Unable to load the image.")
        exit()

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Apply GaussianBlur to reduce noise and help with contour detection
    blurred = cv2.GaussianBlur(gray, (15, 15), 0)  # kernel size
    # Use adaptive thresholding to create a binary image
    _, thresh = cv2.threshold(blurred, 230, 255, cv2.THRESH_BINARY)
    # Find contours in the binary image
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) > 0:
        # Iterate through the contours and find the contour with the largest area (assumed to be the sun)
        largest_contour = max(contours, key=cv2.contourArea)
        # Get the bounding box of the contour
        x, y, w, h = cv2.boundingRect(largest_contour)
        sun_coordinates, sun_radius = (x + w // 2, y + h // 2), max(w // 2, h // 2)

        if sun_radius < 50:
            sun_radius = 50
        if sun_radius > 100:
            sun_radius = 100

        # Draw a yellow circle around the sun
        cv2.circle(image, sun_coordinates, sun_radius, (0, 255, 255), thickness=-1)

    # plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # # plt.title('Processed Image with Sun Detection')
    # plt.axis('off')
    # plt.show()

    return image

# BRBG_segmentation (Semantic)

In [ ]:
def BRBG_segmentation(image_path):
    sun_image = find_sun(image_path)
    # Check if the image was loaded successfully
    sky = 0
    cloud = 0
    if sun_image is None:
        print("Error: Unable to load sun image.")
        exit()

    # Get image dimensions
    height, width, _ = sun_image.shape
    center_x, center_y = width // 2, height // 2
    radius = height // 2

    # Create an empty output image
    output_image = np.zeros_like(sun_image)

    # Iterate through each pixel in the image
    for y in range(height):
        for x in range(width):
            # Get blue, green, and red intensity values for the current pixel
            b = float(sun_image[y, x, 0])
            g = float(sun_image[y, x, 1])
            r = float(sun_image[y, x, 2])

            distance_to_center = np.sqrt((x - center_x)**2 + (y - center_y)**2)

            # Avoid division by zero by adding a small epsilon value
            epsilon = 1e-5

            # Calculate the threshold value for the current pixel
            threshold = (b / (r + epsilon)) + (b / (g + epsilon))

            if b == 0 and r == 0 and g == 0: # Outside the circle
                threshold = 0
            if b == 0 and r == 255 and g == 255: # Sun
                threshold = -1

            # Define the threshold value
            threshold_value = 2.5  # You may adjust this value based on your image characteristics

            # Classify the pixel based on the threshold
            if radius < distance_to_center:
                output_image[y, x, :] = [0, 0, 0]
            elif threshold < 0:
                sky += 1
                output_image[y, x, :] = [0, 255, 255]
            else:
                if threshold > threshold_value:
                    sky += 1
                    # Pixel is above the threshold, set it to blue
                    output_image[y, x, :] = [255, 34, 34]
                else:
                    cloud += 1
                    # Pixel is below the threshold, set it to grey
                    output_image[y, x, :] = [128, 128, 128]
    cloud_cover = (cloud/(sky+cloud)) * 100

    # Display the result using matplotlib
    # plt.imshow(cv2.cvtColor(output_image, cv2.COLOR_BGR2RGB))
    # # plt.title('Processed Image with Sun Detection')
    # plt.axis('off')
    # plt.show()

    return output_image, cloud_cover

In [ ]:
source_folder = "/content/drive/MyDrive/Extracted_2023"
dest_folder = "/content/drive/MyDrive/Image Dataset Curation/Segmentation/2023hourly_new"
date_format = "%Y%m%d%H%M%S"
csv_file_path = '../content/drive/MyDrive/Image Dataset Curation/Segmentation/cloud_cover_hourly_2023_new.csv'

In [ ]:
import os

count = 0
# Iterate directory
for path in os.listdir(source_folder):
    # check if current path is a file
    if os.path.isfile(os.path.join(source_folder, path)):
        count += 1
print('File count:', count)
file_list = []
for filename in os.listdir(source_folder):
    #As picking images with 60 mins time interval to match the numerical dataset
    if "0000_12_UE" in filename:
        file_list.append(filename)

File count: 21305


In [ ]:
len(file_list)

3292

# Saving the segmented images in Folder

In [ ]:
for filename in file_list:
    image_path = os.path.join(source_folder, filename)
    output_image, cloud_cover = BRBG_segmentation(image_path)
    dest_path = os.path.join(dest_folder, filename)
    cv2.imwrite(dest_path, output_image)
    date_string = filename.split("_")[0]
    date_object = datetime.strptime(date_string, date_format)
    date = date_object.strftime("%Y-%m-%d %H:%M:%S")
    with open(csv_file_path, 'a', newline='') as file:
        row = f"{date}\t{cloud_cover}\n"
        file.write(row)